In [ ]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 24.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=49b5b44f01f12e0d622f8a0e381c41e7dcd6cd0e60bbbb8233d6d6df4c46be67
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark= SparkSession.builder.appName('Correlation').getOrCreate()

In [ ]:
from pyspark.ml.stat import Correlation

In [ ]:
df = spark.read.csv('diabetes.csv', inferSchema=True, header=True)

In [ ]:
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [ ]:
df.corr('Pregnancies','Outcome')

0.22189815303398636

In [ ]:
df.corr('Glucose','Outcome')

0.4665813983068737

In [ ]:
df.corr('BloodPressure','Outcome')

0.06506835955033274

In [ ]:
df.corr('SkinThickness','Outcome')

0.07475223191831945

In [ ]:
df.corr('Insulin','Outcome')

0.13054795488404794

In [ ]:
df.corr('BMI','Outcome')

0.2926946626444454

In [ ]:
df.corr('DiabetesPedigreeFunction','Outcome')

0.17384406565296

In [ ]:
df.corr('Age','Outcome')

0.23835598302719757

In [ ]:
df.corr('Outcome','Outcome')

1.0

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
vector_col = "corr-features"

In [ ]:
df.columns

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [ ]:
assembler = VectorAssembler(inputCols=df.columns, outputCol=vector_col)

In [ ]:
df_vector = assembler.transform(df).select(vector_col)

In [ ]:
df_vector

DataFrame[corr-features: vector]

In [ ]:
matrix = Correlation.corr(df_vector,vector_col).collect() [0][0]

In [ ]:
corrmatrix = matrix.toArray().tolist()

In [ ]:
print(corrmatrix)

[[1.0, 0.12945867149927273, 0.1412819774071402, -0.08167177444900704, -0.07353461435162806, 0.017683090727830593, -0.0335226729626132, 0.5443412284023401, 0.22189815303398844], [0.12945867149927273, 1.0, 0.15258958656866442, 0.057327890738176956, 0.3313571099202079, 0.2210710694589828, 0.1373372998283708, 0.2635143198243338, 0.4665813983068757], [0.1412819774071402, 0.15258958656866442, 1.0, 0.2073705384030702, 0.08893337837319268, 0.28180528884991046, 0.04126494793009858, 0.23952794642136369, 0.0650683595503333], [-0.08167177444900704, 0.057327890738176956, 0.2073705384030702, 1.0, 0.4367825701200099, 0.39257320415903724, 0.18392757295416276, -0.11397026236774135, 0.0747522319183196], [-0.07353461435162806, 0.3313571099202079, 0.08893337837319268, 0.4367825701200099, 1.0, 0.1978590564931005, 0.18507092916809895, -0.04216295473537677, 0.13054795488404822], [0.017683090727830593, 0.2210710694589828, 0.28180528884991046, 0.39257320415903724, 0.1978590564931005, 1.0, 0.14064695254510515, 

In [ ]:
df.columns

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [ ]:
columns = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age','Outcome']

In [ ]:
df_corr = spark.createDataFrame(corrmatrix, columns)

In [ ]:
df_corr.show()

+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------------------------+--------------------+-------------------+
|         Pregnancies|             Glucose|      BloodPressure|       SkinThickness|             Insulin|                 BMI|DiabetesPedigreeFunction|                 Age|            Outcome|
+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------------------------+--------------------+-------------------+
|                 1.0| 0.12945867149927273| 0.1412819774071402|-0.08167177444900704|-0.07353461435162806|0.017683090727830593|     -0.0335226729626132|  0.5443412284023401|0.22189815303398844|
| 0.12945867149927273|                 1.0|0.15258958656866442|0.057327890738176956|  0.3313571099202079|  0.2210710694589828|      0.1373372998283708|  0.2635143198243338| 0.4665813983068757|
|  0.1412819774071402| 0.1525895865